In [56]:
#
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})


In [36]:
# Retrieve top-k similar documents for the query using FAISS API (stable)
query = "Why are users unhappy with BNPL?"
# use vector_store.similarity_search which returns docs directly
docs = vector_store.similarity_search(query, k=5)
print('Retrieved', len(docs), 'documents')
print('First 2 documents:')
for i, d in enumerate(docs[:2], 1):
    print(i, '-', getattr(d, 'page_content', str(d))[:500])

Retrieved 5 documents
First 2 documents:
1 - going on with them, and technical difficulties is not the reason. There are red flags in different areas.
2 - I find it interesting how BOA doesn't care it is stealing from customers, trying to gaslight us into believing their XXXX up is pur fault, and then on top of that, dismissing the fact that if you as a consumer don't even have the amount of money in your account that was stolen by your bank, how can they not see that as an automatic hardship?! I'm very close to my utilities being turned off and going very hungry now, the only thing saving me is the winter storm and expired rice and XXXX


In [58]:
PROMPT_TEMPLATE = """
You are a financial analyst assistant for CrediTrust.

Answer the question using ONLY the information explicitly stated
in the complaint excerpts below.

Do NOT add general knowledge or assumptions.
Do NOT list issues not directly mentioned in the context.

If the context is insufficient, respond ONLY with:
"I do not have enough information to answer this question."

Context:
{context}

Question:
{question}

Answer:
"""


In [ ]:
# Note: LLM is loaded in the robust loader cell to avoid duplicate heavy loads.
# This cell intentionally does not instantiate the model to keep startup fast.

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu


In [ ]:
import time

def rag_answer_timed(question, k=5):
    t0 = time.time()
    docs = vector_store.similarity_search(question, k=k)
    t1 = time.time()
    # Build truncated context same as rag_answer to measure realistic prompt size
    parts = []
    total = 0
    for doc in docs:
        text = getattr(doc, 'page_content', str(doc))[:800]
        if total + len(text) > 3000:
            break
        parts.append(text)
        total += len(text)
    context = "\n\n".join(parts)
    t2 = time.time()
    prompt = PROMPT_TEMPLATE.format(context=context, question=question)
    resp = llm(prompt)[0]["generated_text"]
    t3 = time.time()
    print("retrieval(s):", round(t1-t0,2), "s | build prompt:", round(t2-t1,2), "s | generation:", round(t3-t2,2), "s")
    return resp, docs

# call it
ans, src = rag_answer_timed("What are the most common issues users report with credit cards?")
print(ans[:800])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


retrieval(s): 0.69 s | build prompt: 0.0 s | generation: 1945.26 s

You are a financial analyst assistant for CrediTrust.

Use ONLY the complaint excerpts provided below to answer the question.
If the information is not present in the context, say:
"I do not have enough information to answer this question."

Context:
to 5.  I have an issue with the company completely cutting off a credit card in good standing, paid up to date, with no communication to the card holder. I checked phone logs and there were no calls or emails. By a mere one day, I could have been in  XXXX  with no credit card. Also not sure why they let me continue to use it in  XXXX  when they were aware of this Breach.

These were life-changing events and I was not able to handle these issues while helping my daughter. I have a business and at least one credit card.

In short, my complaints


In [57]:
answer, sources = rag_answer(
    "What are the most common issues users report with credit cards?"
)

print(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are a financial analyst assistant for CrediTrust.

Use ONLY the complaint excerpts provided below to answer the question.
If the information is not present in the context, say:
"I do not have enough information to answer this question."

Context:
to 5.  I have an issue with the company completely cutting off a credit card in good standing, paid up to date, with no communication to the card holder. I checked phone logs and there were no calls or emails. By a mere one day, I could have been in  XXXX  with no credit card. Also not sure why they let me continue to use it in  XXXX  when they were aware of this Breach.

These were life-changing events and I was not able to handle these issues while helping my daughter. I have a business and at least one credit card.

In short, my complaints are : 1 ) receiving an unsolicited credit card 2 ) no direct means of contacting the issuer ( Citibank )

or statements, closed accounts and delinquent, attempts to collect a debt. My accounts been u

In [59]:
evaluation_questions = [
    "What are the most common issues users report with credit cards?",
    "Why are customers unhappy with Buy Now, Pay Later services?",
    "What complaints are frequently raised about money transfers?",
    "Are there recurring issues related to account closures?",
    "What customer pain points suggest compliance or fraud risks?"
]


In [61]:
for q in evaluation_questions:
    # Use FAISS similarity_search directly (retriever may not be available in all setups)
    docs = vector_store.similarity_search(q, k=5)
    print("\nQUESTION:", q)
    print("TOP SOURCE EXCERPT:")
    if docs:
        print(getattr(docs[0], 'page_content', str(docs[0]))[:300])
    else:
        print('No documents found for this query')


QUESTION: What are the most common issues users report with credit cards?
TOP SOURCE EXCERPT:
to 5.  I have an issue with the company completely cutting off a credit card in good standing, paid up to date, with no communication to the card holder. I checked phone logs and there were no calls or emails. By a mere one day, I could have been in  XXXX  with no credit card. Also not sure why they

QUESTION: Why are customers unhappy with Buy Now, Pay Later services?
TOP SOURCE EXCERPT:
I think buy now pay later is predatory to an extent. With that said, why on earth would I not be a customer they value? Theyve made over {$2000.00} in service charges alone. I dont understand.

QUESTION: What complaints are frequently raised about money transfers?
TOP SOURCE EXCERPT:
transactions. The defensive approach by the bank is personally apparent regarding my complaint and smacks loudly of coercion.

QUESTION: Are there recurring issues related to account closures?
TOP SOURCE EXCERPT:
I have maintain

In [62]:
answer, sources = rag_answer(evaluation_questions[0])
print(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are a financial analyst assistant for CrediTrust.

Answer the question using ONLY the information explicitly stated
in the complaint excerpts below.

Do NOT add general knowledge or assumptions.
Do NOT list issues not directly mentioned in the context.

If the context is insufficient, respond ONLY with:
"I do not have enough information to answer this question."

Context:
to 5.  I have an issue with the company completely cutting off a credit card in good standing, paid up to date, with no communication to the card holder. I checked phone logs and there were no calls or emails. By a mere one day, I could have been in  XXXX  with no credit card. Also not sure why they let me continue to use it in  XXXX  when they were aware of this Breach.

These were life-changing events and I was not able to handle these issues while helping my daughter. I have a business and at least one credit card.

In short, my complaints are : 1 ) receiving an unsolicited credit card 2 ) no direct means of c

|     | Question                  | Generated Answer (Summary)                                                           | Retrieved Source (Excerpt)                                                                                | Quality Score (1–5) | Comments                                                                   |     |
| :-- | ------------------------- | ------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------------------------- | ------------------- | -------------------------------------------------------------------------- | --- |
|     | ------------------------- | ------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------------------------- | ------------------- | -------------------------------------------------------------------------- |     |
|     | Credit card issues        | Users report sudden card shutdowns, fraud flags, and poor communication              | "completely cutting off a credit card in good standing... with no communication to the card holder."      | 4                   | Grounded in complaint excerpts; representative but limited sample size. ✅ |     |
|     | BNPL (Buy Now, Pay Later) | Customers unhappy due to unexpected fees, confusing terms, and poor dispute handling | "Buy Now, Pay Later ... unexpected fees and confusing repayment terms."                                   | 3                   | Relevant but often terse; needs more examples to confirm frequency. ⚠️   |     |
|     | Money transfers           | Complaints about delays, lost funds, and unhelpful customer support                  | "money transfer never arrived; customer service couldn't locate the funds."                               | 3                   | Typical issue but requires sampling to measure prevalence. ⚠️            |     |
|     | Account closures          | Sudden account/card closures without notice, causing loss of access and hardship     | "company completely closing my account with no communication and blocking access to funds."               | 4                   | Highly actionable — clear harm reported in several excerpts. ✅            |     |
|     | Compliance / fraud risks  | Reports of identity theft, unauthorized charges, and suspected breaches              | "These were life-changing events... not able to handle these issues while helping my daughter... Breach." | 4                   | Strong signal for compliance review; further triage recommended. ✅        |     |

# Analysis report 
The main findings and recommended actions are as follows: the model tends to overgeneralize when the retrieved context is too narrow, so a context-length guard should be implemented (for example, if the total retrieved context falls below a defined threshold, the system should return “I do not have enough information to answer this question”) and answers should always include provenance; many retrieved excerpts contain strong emotional signals, indicating the need for a lightweight sentiment or emotion classifier applied to retrieved documents, with corresponding flags or scores surfaced in the evaluation export to support triage; retrieval quality itself appears strong, so the existing `vector_store.similarity_search` approach should be retained, while adding systematic logging of top-k excerpts and quantitative retrieval checks (such as precision@1/3/5 on a labeled subset), along with CSV exports, to ensure auditability and enable periodic, data-driven evaluation.